In [1]:
import gurobipy as gp
from gurobipy import GRB
import math

# Classification

In [25]:
def cost(L, B, Bi, l0, l1, l2):
    return (L + 2*l2)*(B-Bi)*(B-Bi)/2 + l0*(Bi!=0) + l1*abs(Bi)


def calculate_optimal_gurobi(L=None, l0=None, l1=None, l2=None, B=None, lb=None, ub=None, M=100):
    m = gp.Model('box_constraint_test')
    m.setParam( 'OutputFlag', False )
    m.setParam("MIPGap", 1e-10)
    m.setParam("MIPGapAbs", 1e-6)
    # Variables
    
    Bi = m.addVar(lb=lb, ub=ub, vtype=GRB.CONTINUOUS, name="Bi")
    Bi_supp = m.addVar(vtype=GRB.BINARY, name = '1[Bi]')
    Bi_sign = m.addVar(vtype=GRB.BINARY)
    Bi_pos = m.addVar(lb=0, ub=ub, vtype=GRB.CONTINUOUS)
    Bi_neg = m.addVar(lb=0, ub=-lb, vtype=GRB.CONTINUOUS)
    
    #Constraints
    m.addConstr(Bi_pos <= M*Bi_sign)
    m.addConstr(Bi_neg <= M*(1-Bi_sign))
    m.addConstr(Bi == Bi_pos - Bi_neg)

    m.addConstr(Bi >= -M*Bi_supp)
    m.addConstr(Bi <= M*Bi_supp)
    
    #Cost
    m.setObjective((L+2*l2)*(B - Bi)*(B - Bi)/2 
                   + l0*Bi_supp
                   + l1*(Bi_pos + Bi_neg), GRB.MINIMIZE)
    
    m.optimize()
    
    for v in m.getVars():
        if v.varName == "Bi":
            return v.x

In [26]:
def box(x, lb, ub):
    if x > ub:
        return ub
    elif x <= lb:
        return lb
    else:
        return x
    
def sign(x):
    if x < 0:
        return -1
    return 1


def calculate_closed_form_Lipschitz(L=None, l0=None, l1=None, l2=None, 
                                    B=None, lb=None, ub=None, M=None):
    Bi = B # Attempt to make Bi, B. This is solved for iteratively using other approaches
    
    if (abs(Bi) - l1/(L+2*l2)) < math.sqrt(2*l0/(L + 2*l2)):
        return 0
    
    x = box(sign(Bi)*(abs(Bi) - l1/(L+2*l2)), lb, ub)
    
    delta = math.sqrt((abs(Bi) - l1/(L+2*l2))**2 - 2*l0/(L+2*l2))
    
    low = sign(Bi)*(abs(Bi) - l1/(L+2*l2)) - delta
    high = sign(Bi)*(abs(Bi) - l1/(L+2*l2)) + delta
#     print("x box", x)
#     print(f"range: [{low}, {high}]")
#     print(f"range no delta: [{low+delta}, {high-delta}]")
    print("low", low)
    print('high', high)
    print('x', x)
    if low <= x <= high:
        return x
    else:
        return 0

In [33]:
L = 2
l0=0
l1=11.2325
l2=0
B=5.62674
lb=-float('inf')
ub=float('inf')

In [34]:
Bi_optimal = calculate_optimal_gurobi(L=L, l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub, M = 1000)
print(Bi_optimal, cost(L=L, B=B, Bi=Bi_optimal, l0=l0, l1=l1, l2=l2))

0.010489999999999888 31.660092987499997


In [35]:
Bi_box = calculate_closed_form_Lipschitz(L=L, l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
print(Bi_box, cost(L=L, B=B, Bi=Bi_box, l0=l0, l1=l1, l2=l2))

low 0.0
high 0.020979999999999777
x 0.010489999999999888
0.010489999999999888 31.660092987499997


# Lets run on our problem

In [7]:
import hypothesis
from hypothesis import given, settings, assume, example
from hypothesis.strategies import floats
import numpy as np

# Compare values

In [14]:
@given(L=floats(.1, 1), l0=floats(0, 10), l1 = floats(0, 10), l2 = floats(0, 10),
       B=floats(-10, 10), lb=floats(-1, 0), ub=floats(0, 1))
@settings(max_examples=10000)
@example(L=0.25, l0=0, l1=0, l2=0, B=1, lb=-1, ub=1)
@example(L=0.25, l0=0, l1=0, l2=10, B=1, lb=-1, ub=1)
@example(L=0.25, l0=0, l1=10, l2=0, B=1, lb=-1, ub=1)
@example(L=0.25, l0=0, l1=10, l2=10, B=1, lb=-1, ub=1)
@example(L=0.25, l0=10, l1=0, l2=10, B=1, lb=-1, ub=1)
@example(L=0.25, l0=10, l1=10, l2=0, B=1, lb=-1, ub=1)
@example(L=0.25, l0=10, l1=10, l2=10, B=1, lb=-1, ub=1)
@example(L=0.25, l0=0, l1=0, l2 =0,B =-0.1, lb=-0.02, ub=0)
@example(L=0.25, l0=0.0061658496635019775, l1=0.0, l2=1e-4, B=4, lb=0.0, ub=0.0015000000000000573)
def compare_B(L, l0, l1, l2, B, lb, ub):
    gurobi = calculate_optimal_gurobi(L=L, l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub, M = 1000)
    gurobi = cost(L, B, gurobi, l0, l1, l2)
    closed = calculate_closed_form_Lipschitz(L=L, l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
    closed = cost(L, B, closed, l0, l1, l2)
    
    if gurobi <= closed:
        np.testing.assert_almost_equal(gurobi, closed)
    


In [15]:
compare_B()